In [1]:
import re #regex
import os #operating system
import pandas as pd
import numpy as np

In [2]:
directory = 'csv/perf_only_algorithm'
files = []
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if (re.match("^fwa_dev_v_.*", filename)) :
        files.append(file_path)
        print(file_path)

csv/perf_only_algorithm/fwa_dev_v_1_1__n_50__b_10__t_1.csv
csv/perf_only_algorithm/fwa_dev_v_1_2__n_50__b_10__t_1.csv
csv/perf_only_algorithm/fwa_dev_v_1_3__n_50__b_10__t_1.csv
csv/perf_only_algorithm/fwa_dev_v_1_4__n_50__b_10__t_1.csv
csv/perf_only_algorithm/fwa_dev_v_2_0__n_50__b_10__t_1.csv
csv/perf_only_algorithm/fwa_dev_v_2_2__n_50__b_10__t_1.csv


In [3]:
# for filename in files :
#     f = open(filename, "r")
#     print(f.read())
#     f.close()

In [4]:
def obtain_parameter(in_string, par_name) :
    par_regex = "-" + par_name + "=[^ \n]*"
    par_string = re.search(par_regex, in_string).group()
    par_value = re.sub(".*=", "", par_string)
    return par_value

def obtain_version(in_string) :
    par_regex = "v_[^. ]*"
    par_string = re.search(par_regex, in_string).group()
    par_value = re.sub("v_", "", par_string)
    return par_value

In [5]:
def obtain_all_parameters(in_file) :
    with open(in_file) as fp :
        for i, line in enumerate(fp) :
            if i == 0 :
                n = int(obtain_parameter(line, "n"))
                b = int(obtain_parameter(line, "b"))
                t = int(obtain_parameter(line, "t"))
                version = obtain_version(line)

            elif i == 3 :
                headers = line.replace('"', '').replace("\n", "").split(",")

            elif i == 4 :
                temp = line.replace("\n", "").replace("%", "").split(",")
                unit_measures = []
                for unit in temp :
                    if unit != "" :
                        unit = "(" + unit + ")"
                    unit_measures.append(unit)

            elif i > 4 :
                break

    headers = list(map(lambda x, y: x + y, headers, unit_measures))

    return n, b, t, version, headers

In [6]:
df_big = pd.DataFrame()     # collects all data
df_small = pd.DataFrame()   # sums up everything
for filename in files : 
    n, b, t, version, headers = obtain_all_parameters(filename)
    temp_df_big = pd.read_csv(filename, skiprows=5, names=headers)
    temp_df_big['version'] = version
    temp_df_big['n'] = n
    temp_df_big['b'] = b
    temp_df_big['t'] = t
    if df_small.empty :
        df_big = pd.DataFrame(columns = headers)
        df_small = pd.DataFrame(columns = ['version', 'n', 'b', 't', 'Time(us)'])
    df_big = pd.concat([df_big, temp_df_big])
    tot_time = temp_df_big['Time(us)'].sum()
    #print(f"version = {version}, n = {n}, b = {b}, t = {t}, tot_time = {tot_time:.2f}")
    df_small.loc[-1] = [version, n, b, t, tot_time]  # adding a row
    df_small.index = df_small.index + 1  # shifting index
    df_small = df_small.sort_index()  # sorting by index
df_small.sort_values(by = ['n', 'b', 't', 'Time(us)'])


,version,n,b,t,Time(us)
1,2_0,50,10,1,2149.167
0,2_2,50,10,1,2281.017
3,1_3,50,10,1,2324.565
4,1_2,50,10,1,2390.845
2,1_4,50,10,1,3175.155
5,1_1,50,10,1,3731.109


In [7]:
na_columns = df_big.columns[df_big.isna().any()].tolist()
if na_columns == [] :
    print("No NaN values found")
else :
    print("Columns with NaN:")
    print(na_columns)

No NaN values found
